In [10]:
%pip install fastapi nest-asyncio pyngrok uvicorn jinja2 numpy scikit-learn requests

In [2]:
import pickle
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from fastapi import FastAPI, Request
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
import nest_asyncio
import uvicorn
from pyngrok import ngrok
from pydantic import BaseModel
from starlette.responses import FileResponse
import requests
import json

In [3]:
class IrisSpecies(BaseModel):
  sepal_length: float
  sepal_width: float
  petal_length: float
  petal_width: float

In [4]:
app = FastAPI()

In [5]:
@app.get('/')
async def sample():
  loaded_X = pickle.load(open('X_train.pkl', 'rb'))

  x0 = np.random.normal(loaded_X[:, 0].mean(), loaded_X[:, 0].std(), 1)
  x1 = np.random.normal(loaded_X[:, 1].mean(), loaded_X[:, 1].std(), 1)
  x2 = np.random.normal(loaded_X[:, 2].mean(), loaded_X[:, 2].std(), 1)
  x3 = np.random.normal(loaded_X[:, 3].mean(), loaded_X[:, 3].std(), 1)

  return {
    'sepal_length': x0[0],
    'sepal_width': x1[0],
    'petal_length': x2[0],
    'petal_width': x3[0]
    }

In [6]:
@app.post('/')
async def predict(iris: IrisSpecies):
  data = iris.dict()
  loaded_model = pickle.load(open('model.pkl', 'rb'))

  data_in = [[data['sepal_length'], data['sepal_width'], data['petal_length'], data['petal_width']]]
  prediction = loaded_model.predict(data_in)
  probability = loaded_model.predict_proba(data_in).max()
  iris_type = ['Iris Setosa', 'Iris Virginica', 'Iris Versicolor']
  
  return {
    'prediction': int(prediction[0]),
    'iris_specie': iris_type[int(prediction[0])-1],
    'probability': probability
  }

In [7]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

t=2021-12-10T00:34:15-0300 lvl=warn msg="can't bind default web address, trying alternatives" obj=web addr=127.0.0.1:4040
INFO:     Started server process [14572]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: http://6593-2804-14d-b086-5343-7cb5-f57a-ccb2-d577.ngrok.io


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [14572]


In [9]:
# %pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# version = !python --version
# version = version[0].lower().replace(" ","-")
# !echo $version > runtime.txt

In [ ]:
# %echo web: voila --port=$PORT --no-browser api-colab.ipynb > Procfile